In [2]:
import os

from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI

os.environ["OPENAI_API_VERSION"] = "2024-02-15-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://openaimodelv3si.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = "e248b8e8dbf94849ac6888971cacc9ae"

In [3]:
llm = AzureChatOpenAI(
    openai_api_version="2023-05-15",
    azure_deployment="trial1",
)

In [ ]:
message = HumanMessage(
    content="Translate this sentence from English to French. I love programming."
)
llm.invoke([message])

In [5]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/Users/aryan/Desktop/fargowells/mlpoc/hackathon/sustainability-survey-automation/training-data/2021-annual-report.pdf")
pages = loader.load_and_split()
# pages

In [6]:
# from langchain_openai import OpenAIEmbeddings
# embeddings = OpenAIEmbeddings()

from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = AzureOpenAIEmbeddings(
    azure_deployment="embed1",
    openai_api_version=os.getenv("OPENAI_API_VERSION"),
)
vector = FAISS.from_documents(pages, embeddings)

In [ ]:
# from langchain_core.prompts import ChatPromptTemplate
# # from langchain_community.document_chains import create_stuff_documents_chain
# # from langchain_community.retrieval_chains import create_retrieval_chain
# from langchain.chains.combine_documents import create_stuff_documents_chain
# from langchain_core.runnables import RunnablePassthrough

# # Create retriever
# retriever = vector.as_retriever()

# # Create Agent
# # llm = Ollama(model="llama2")
# prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context in a short and concise manner. Do not use any external information. The context is as follows:

# <context>
# {context}
# </context>

# Question: {input}""")
# document_chain = create_stuff_documents_chain(llm, prompt)

# # retrieval_chain = create_retrieval_chain(retriever, document_chain)

# def format_docs(docs):
#     return "\n\n".join(doc.page_content for doc in docs)

# rag_chain = (
#     {"context": retriever | format_docs, "question": RunnablePassthrough()}
#     | prompt
#     | llm
# )
# rag_chain.invoke({"question": "summarize this annual report"})

In [7]:
from langchain.chains import RetrievalQA
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)

review_template = """Your job is to use ESG (A sustainability report is a report published by companies on the environmental, social and governance (ESG) impacts of their activities) 
documents and annual reports to answer questions. Use
the following context to answer questions. Be as detailed as possible, but
don't make up any information that's not from the context. If you don't know
an answer, say you don't know.
{context}
"""

esg_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["context"], template=review_template)
)

esg_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["question"], template="{question}")
)
messages = [esg_system_prompt, esg_human_prompt]

esg_prompt = ChatPromptTemplate(
    input_variables=["context", "question"], messages=messages
)

retriever = vector.as_retriever()

esg_vector_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)
# esg_vector_chain.combine_documents_chain.llm_chain.prompt = esg_prompt

In [8]:
query = "Identify the position(s) (do not include any names) of the individual(s) on the board with responsibility for climate-related issues."
response = esg_vector_chain.invoke(query)
response

{'query': 'Identify the position(s) (do not include any names) of the individual(s) on the board with responsibility for climate-related issues.',
 'result': 'The position responsible for climate-related issues on the board is the Corporate Responsibility Committee.',
 'source_documents': [Document(page_content='Board of Directors \nSteven D. Black Maria R. Morris \nChairman, Retired Executive Vice President and Head of \nWells Fargo & Company Global Employee Benefits business, MetLife, Inc. \nMark A. Chancy Richard B. Payne, Jr. \nRetired Vice Chairman and Co-Chief Operating Retired Vice Chairman, Wholesale Banking, \nOfficer, SunTrust Banks, Inc. U.S. Bancorp \nCeleste A. Clark Juan A. Pujadas \nPrincipal, Abraham Clark Consulting, LLC, and Retired Senior Retired Principal, PricewaterhouseCoopers LLP, \nVice President, Global Public Policy and External Relations and former Vice Chairman, Global Advisory Services, \nand Chief Sustainability Officer, Kellogg Company PwC International \